# Linking Movie Titles in Multiple Data Tables


## String Matching approach

In [ ]:
%%bash
#pip install fuzzywuzzy
#pip install python-Levenshtein

In [1]:
from fuzzywuzzy import process
from unidecode import unidecode

In [2]:
import pandas as pd
from ast import literal_eval
import numpy as np
import os

def clean(row):
    '''apply on rows of a dataframe to clean up'''
    #this is because CSV conversion has converted list into string
    row['plots'] = literal_eval(row['plots'])
    try:
        row['year'] = int(row['year'])
        return row
    except:
        row['year'] = np.nan
        return row

# TODO: Set this appropriately
dropbox_dir = "C:\\Users\\sooraj.raveendra\\Dropbox\\moviemeta\\"

movieplots = pd.read_csv(dropbox_dir + 'movieplots.csv')
movieplots = movieplots.apply(clean, axis = 1)

In [12]:
'''
    The do_fuzzy parameter controls whether the approximate string match algorithm (Levenshtein algorithm)
    is used or not.
    
    WARNING: doing fuzzy search can take a LOT of time
'''
def create_title_map(imdb_titles, wiki_titles, year, do_fuzzy=False):
    titledict = {}
    for wiki_title in wiki_titles:
        match = next((s for s in list(imdb_titles) if s == wiki_title + " (%d)" % year), None)
        if(match is None and do_fuzzy):
            fuzzy_match, similarity_pct = process.extractOne(wiki_title + " (%d)" % year, imdb_titles)
            if(similarity_pct >= 90): # Take only the ones with high similarity
                match = fuzzy_match

        if(match is not None):
            titledict[wiki_title] = match
    
    return titledict
            
yearly_maps = {}
for year in range(2010, 2015):
    imdbdf = movieplots[movieplots.year == year]
    imdb_titles = imdbdf.title.apply(lambda x: unidecode(x))
    wikidf = pd.read_table(os.path.join(dropbox_dir, "wikipedia", "wikipedia_plots_%d.csv" % year))
    yearly_maps[year] = create_title_map(imdb_titles, wikidf.title, year, False)

E:\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:24: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.


In [13]:
yearly_maps[2013]

{'11.6': '11.6 (2013)',
 '12 Years a Slave': '12 Years a Slave (2013)',
 '2 Guns': '2 Guns (2013)',
 '21 & Over': '21 & Over (2013)',
 '3 Geezers!': '3 Geezers! (2013)',
 '42': '42 (2013)',
 '47 Ronin': '47 Ronin (2013)',
 '6-5=2': '6-5=2 (2013)',
 '7 Assassins': '7 Assassins (2013)',
 '9 Full Moons': '9 Full Moons (2013)',
 "A Birder's Guide to Everything": "A Birder's Guide to Everything (2013)",
 'A Farewell to Fools': 'A Farewell to Fools (2013)',
 'A Field in England': 'A Field in England (2013)',
 'A Good Day to Die Hard': 'A Good Day to Die Hard (2013)',
 'A Haunted House': 'A Haunted House (2013)',
 'A Madea Christmas': 'A Madea Christmas (2013)',
 'A Mile from Home': 'A Mile from Home (2013)',
 'A Moment of Love': 'A Moment of Love (2013)',
 'A Night in Old Mexico': 'A Night in Old Mexico (2013)',
 'A Promise': 'A Promise (2013)',
 'A Resurrection': 'A Resurrection (2013)',
 'A River Changes Course': 'A River Changes Course (2013)',
 'A Spell to Ward Off the Darkness': 'A Spel

## Using dbpedia and linkedmdb

In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import re
import urllib2
from bs4 import BeautifulSoup
import httplib
import time

unused1 = """
                PREFIX dbpont: <http://dbpedia.org/ontology/>
                PREFIX dbpprop: <http://dbpedia.org/property/>
                PREFIX dbres: <http://dbpedia.org/resource/>
                PREFIX dbc: <http://dbpedia.org/resource/Category:>
                PREFIX foaf: <http://xmlns.com/foaf/0.1/>
                PREFIX dcterms: <http://purl.org/dc/terms/>
                PREFIX wdentity: <http://wikidata.org/entity/>
                PREFIX lmdb: <http://data.linkedmdb.org/resource/>
                PREFIX wd: <http://www.wikidata.org/entity/>
                PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                PREFIX wikibase: <http://wikiba.se/ontology#>

                SELECT DISTINCT ?title 
                       (group_concat(distinct ?language;separator=", ") as ?languages) 
                       (group_concat(distinct ?country;separator=", ") as ?countries) 
                       ?released
                       ?gross
                       ?comment
                       ?abstract
                       (?movie as ?dbpediaLink)
                       ?wikipediaLink
                       ?wikidataID
                       ?imdb
                       ?rt
                WHERE
                {
                        ?movie rdf:type dbpont:Film .
                        ?movie rdfs:label ?title .
                        ?movie rdfs:comment ?comment .
                        ?movie dbpont:abstract ?abstract .
                        ?movie dcterms:subject dbc:%(year)s_films . 
                        OPTIONAL { ?movie dbpprop:released ?released }
                        OPTIONAL { ?movie dbpprop:country ?country } 
                        OPTIONAL { ?movie dbpont:gross ?gross }
                        OPTIONAL { ?movie dbpprop:language ?language }
                        OPTIONAL { ?movie foaf:isPrimaryTopicOf ?wikipediaLink }
                        OPTIONAL { ?movie owl:sameAs ?wikidataID }
                        OPTIONAL { ?wikidataID wdt:P345 ?imdb } 
                        OPTIONAL { ?wikidataID wdt:P1258 ?rt }

                        FILTER (lang(?title) = 'en')
                        FILTER (lang(?abstract) = 'en')
                        FILTER (lang(?comment) = 'en')
                        FILTER regex(?wikidataID, "wikidata.org/entity")
                }
                """

movies_for_year_query = """
                PREFIX dbpont: <http://dbpedia.org/ontology/>
                PREFIX dbpprop: <http://dbpedia.org/property/>
                PREFIX dbres: <http://dbpedia.org/resource/>
                PREFIX dbc: <http://dbpedia.org/resource/Category:>
                PREFIX foaf: <http://xmlns.com/foaf/0.1/>
                PREFIX dcterms: <http://purl.org/dc/terms/>
                PREFIX wdentity: <http://wikidata.org/entity/>
                PREFIX wd: <http://www.wikidata.org/entity/>
                PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                PREFIX wikibase: <http://wikiba.se/ontology#>

                SELECT DISTINCT ?title 
                       (group_concat(distinct ?language;separator=", ") as ?languages) 
                       (group_concat(distinct ?country;separator=", ") as ?countries) 
                       ?released
                       ?gross
                       ?comment
                       ?abstract
                       (?movie as ?dbpediaLink)
                       ?wikipediaLink
                       ?wikidataID

                WHERE
                {
                        ?movie rdf:type dbpont:Film .
                        ?movie rdfs:label ?title .
                        ?movie rdfs:comment ?comment .
                        ?movie dbpont:abstract ?abstract .
                        ?movie dcterms:subject dbc:%(year)s_films . 
                        OPTIONAL { ?movie dbpprop:released ?released }
                        OPTIONAL { ?movie dbpprop:country ?country } 
                        OPTIONAL { ?movie dbpont:gross ?gross }
                        OPTIONAL { ?movie dbpprop:language ?language }
                        OPTIONAL { ?movie foaf:isPrimaryTopicOf ?wikipediaLink }
                        OPTIONAL { ?movie owl:sameAs ?wikidataID }

                        FILTER (lang(?title) = 'en')
                        FILTER (lang(?abstract) = 'en')
                        FILTER (lang(?comment) = 'en')
                        FILTER regex(?wikidataID, "wikidata.org/entity")
                }
                """

unused2 = """
                PREFIX dbpont: <http://dbpedia.org/ontology/>
                PREFIX dbpprop: <http://dbpedia.org/property/>
                PREFIX dbres: <http://dbpedia.org/resource/>
                PREFIX dbc: <http://dbpedia.org/resource/Category:>
                PREFIX foaf: <http://xmlns.com/foaf/0.1/>
                PREFIX dcterms: <http://purl.org/dc/terms/>
                PREFIX wdentity: <http://wikidata.org/entity/>

                SELECT DISTINCT ?title 
                       (group_concat(distinct ?language;separator=", ") as ?languages) 
                       (group_concat(distinct ?country;separator=", ") as ?countries) 
                       ?released
                       ?gross
                       ?comment
                       ?abstract
                       (?movie as ?dbpediaLink)
                       ?wikipediaLink
                       ?wikidataID

                WHERE
                {
                        ?movie rdf:type dbpont:Film .
                        ?movie rdfs:label ?title .
                        ?movie rdfs:comment ?comment .
                        ?movie dbpont:abstract ?abstract .
                        ?movie dcterms:subject dbc:%(year)s_films . 
                        OPTIONAL { ?movie dbpprop:released ?released }
                        OPTIONAL { ?movie dbpprop:country ?country } 
                        OPTIONAL { ?movie dbpont:gross ?gross }
                        OPTIONAL { ?movie dbpprop:language ?language }
                        OPTIONAL { ?movie foaf:isPrimaryTopicOf ?wikipediaLink }

                        FILTER (lang(?title) = 'en')
                        FILTER (lang(?abstract) = 'en')
                        FILTER (lang(?comment) = 'en')
                }
                """


def sparql_json_to_df(results, year):
    movie_dicts = []

    for movie in results['results']['bindings']:
        title = movie['title']['value']
        languages = movie.get('languages', None)
        if(languages):
            languages = languages['value']
        countries = movie.get('countries', None)
        if(countries):
            countries = countries['value']
        released = movie.get('released', None)
        if(released):
            released = released['value']
        gross = movie.get('gross', None)
        if(gross):
            gross = gross['value']
        comment = movie.get('comment', None)
        if(comment):
            comment = comment['value']
        abstract = movie.get('abstract', None)
        if(abstract):
            abstract = abstract['value']
        dbpediaLink = movie.get('dbpediaLink', None)
        if(dbpediaLink):
            dbpediaLink = dbpediaLink['value']
        wikipediaLink = movie.get('wikipediaLink', None)
        if(wikipediaLink):
            wikipediaLink = wikipediaLink['value']

        wikidataID = movie.get('wikidataID', None)
        if(wikidataID):
            wikidataID = wikidataID['value']
            
        movie_dicts.append({'year':year,
                            'title':title,
                            'languages':languages,
                            'countries':countries,
                            'released':released,
                            'gross':gross,
                            'comment':comment,
                            'abstract':abstract,
                            'dbpediaLink':dbpediaLink,
                            'wikipediaLink':wikipediaLink,
                            'wikidataID': wikidataID})

    df = pd.DataFrame(movie_dicts, columns=['year'] + results['head']['vars'])
    return(df)

def get_dbpedia_data_for_year(year):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(movies_for_year_query % {'year': str(year)})
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    year_df = sparql_json_to_df(results, year)
    # Do some cleanup
    strip_url = lambda x: re.sub("http://dbpedia.org/resource/", "", x)
    year_df.languages = year_df.languages.apply(strip_url)
    year_df.languages = year_df.languages.apply(lambda x: re.sub("_language", "", x))
    year_df.countries = year_df.countries.apply(strip_url)
    # Remove (film) or (<year> film) at the end of the title
    year_df.title = year_df.title.apply(lambda x: re.sub(r"[\ ]*\((%s)?[\ ]*film\)" % str(year), "", x))
    return(year_df)


In [2]:
years = range(2000, 2016)

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(movies_for_year_query % {'year': str(2001)})
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

#for year in years:
#year = 2010
#dbp_df = get_dbpedia_data_for_year(year)
    #dbp_df['wiki_plot'] = dbp_df.wikipediaLink.apply(lambda link:get_wikipedia_plot(link))
    #csv_filename = "wikipedia_plots_" + str(year) + ".csv"
    #dbp_df.to_csv(csv_filename, sep='\t', encoding='utf-8')
    #print(str(len(dbp_df)), " records for the year ", str(year), "written to ", csv_filename)

In [77]:
def parse_url(row):
    try:
        url = dict(row)['value']
    except:
        url = None
    return url

dbp_df.wikidataID = dbp_df.wikidataID.apply(parse_url)

In [59]:
wdquery = """
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX wikibase: <http://wikiba.se/ontology#>

            SELECT ?imdb ?rt WHERE {
               wd:%s wdt:P345 ?imdb .
               OPTIONAL { wd:%s wdt:P1258 ?rt }
            }

            """

def get_imdb_title(wikidata_url):
    print(wikidata_url)
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    wikidata_id = wikidata_url.rsplit('/',1)[-1]
    sparql.setQuery(wdquery % (wikidata_id, wikidata_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #year_df = sparql_json_to_df(results, year)
    return results

dbp_df.wikidataID[:3].apply(get_imdb_title)

NameError: name 'dbp_df' is not defined